<a href="https://colab.research.google.com/gist/MarkCreightonQueens/e9a49e896b7dafb8d1e4ea4eccd118a1/machine-learning-training-multinomial-naive-bayes-0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random as r
r.seed(0)

In [ ]:
#installing tools
!pip install transformers 
!pip install wget
!pip install tika
!pip install ijson

     |████████████████████████████████| 757kB 6.4MB/s 
     |████████████████████████████████| 890kB 30.6MB/s 
     |████████████████████████████████| 1.1MB 35.3MB/s 
     |████████████████████████████████| 3.0MB 42.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=e60654008f5d10b224256168bd7952df077c208effd78d51c838b55a1adec46b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=0e3ea286ef13eb270b7aa9736bb27629af5f1963344bb28cc44e9b1be00550a1
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
  Created wheel for tika: filename=tika-1.24-cp36-none-any.whl size=32885 sha256=2040acbbe59d19e6e4090ca57e3c1e486e54dd1c2f8e05eda2e18cbe76ede165
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b

In [ ]:
#Importing tools
import pandas as pd
import numpy as np
from tika import parser
%tensorflow_version 1.x
import tensorflow as tf
import torch
import os
import shutil
import glob
import nltk
nltk.download('punkt')
import wget
import urllib.request
import re
import gzip
from nltk.tokenize import sent_tokenize
import json

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

TensorFlow 1.x selected.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#get GPU device name
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
  print('Found GPU at:{}'.format(device_name))
else:
  raise SystemError('GPU device not found')

Found GPU at:/device:GPU:0


In [ ]:
#Determining what GPU is available for use
if torch.cuda.is_available():
  device = torch.device("cuda")
  print ('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))
else: 
  print('No GPU available, using CPU instead.')
  device = torch.device("CPU")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
shutil.unpack_archive('/content/drive/My Drive/Thesis/yelp_review_full_csv.tgz')

In [ ]:
#Reading in the CSV. 

data_train_full = pd.read_csv('/content/yelp_review_full_csv/train.csv', header = None)
data_test = pd.read_csv('/content/yelp_review_full_csv/test.csv', header = None)

print(data_train_full.shape)
print(data_test.shape)

#Creating a smaller subset of the training dataset due to computational restrictions
data_train_1 = data_train_full[data_train_full[0] == 1].sample(50000, random_state=0)
data_train_2 = data_train_full[data_train_full[0] == 2].sample(50000, random_state=0)
data_train_3 = data_train_full[data_train_full[0] == 3].sample(50000, random_state=0)
data_train_4 = data_train_full[data_train_full[0] == 4].sample(50000, random_state=0)
data_train_5 = data_train_full[data_train_full[0] == 5].sample(50000, random_state=0)
frames = [data_train_1, data_train_2, data_train_3, data_train_4, data_train_5]
data_train = pd.concat(frames)
#randomizing the order of the observations
data_train = data_train.sample(frac=1, random_state=0)

print(data_train[0].unique())
print(data_train.shape)

#Reassigning values so that 1-3 = 0 and 4-5 = 1
data_train[0] = data_train[0].replace(1,0)
data_train[0] = data_train[0].replace(2,0)
data_train[0] = data_train[0].replace(3,0)
data_train[0] = data_train[0].replace(4,1)
data_train[0] = data_train[0].replace(5,1)

data_test[0] = data_test[0].replace(1,0)
data_test[0] = data_test[0].replace(2,0)
data_test[0] = data_test[0].replace(3,0)
data_test[0] = data_test[0].replace(4,1)
data_test[0] = data_test[0].replace(5,1)

print(data_train[0].unique())
data_train = data_train.rename(columns={0:'rating',1:'text'})
data_test = data_test.rename(columns={0:'rating',1:'text'})
data_test



(650000, 2)
(50000, 2)
[3 2 4 1 5]
(250000, 2)
[0 1]


,rating,text
0,0,I got 'new' tires from them and within two wee...
1,0,Don't waste your time. We had two different p...
2,0,All I can say is the worst! We were the only 2...
3,0,I have been to this restaurant twice and was d...
4,0,Food was NOT GOOD at all! My husband & I ate h...
...,...,...
49995,0,Just wanted to write a review to chip in with ...
49996,1,Great ambience. Great drinks. Great food. I lo...
49997,1,I have been to the other Monks locations so I ...
49998,0,Don't go here. I know you might want to try i...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
MNB_0_1_data = pd.read_csv('/content/drive/My Drive/Thesis/Output_backup.csv')
print(MNB_0_1_data.shape)

cv = CountVectorizer(binary=True)
cv.fit(data_train['text'])
X = cv.transform(data_train['text'])
X_test = cv.transform(data_test['text'])
MNB_0_1_vector = cv.transform(MNB_0_1_data['Sentences'])


(42083, 15)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = pd.Series(data_train['rating'])

In [ ]:
model = MultinomialNB(alpha = 1, fit_prior = True)
model.fit(X, target)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [ ]:
target_test = pd.Series(data_test['rating'])

In [ ]:
print ("Final Accuracy with laplace smoothing:", accuracy_score(target_test, model.predict(X_test)))

Final Accuracy with laplace smoothing: 0.82724


In [ ]:
model = MultinomialNB(alpha = 1.0e-10, fit_prior = True)
model.fit(X, target)

MultinomialNB(alpha=1e-10, class_prior=None, fit_prior=True)

In [ ]:
target_test = pd.Series(data_test['rating'])

In [ ]:
print ("Final Accuracy without laplace smoothing:", accuracy_score(target_test, model.predict(X_test)))

Final Accuracy without laplace smoothing: 0.79842


In [ ]:
MNB_0_1_results = model.predict(MNB_0_1_vector)
print(MNB_0_1_results.shape)

(42083,)


In [ ]:
os.chdir('/content/drive/My Drive/Thesis/Multinomial Naive Bayesian/0_1')
MNB_0_1_results = pd.DataFrame(data = MNB_0_1_results)
outfile = 'MNB_0_1_results.csv'
MNB_0_1_results.to_csv(outfile)

In [ ]:
import pickle
os.chdir('/content/drive/My Drive/Thesis/Multinomial Naive Bayesian/0_1')
filename = 'Multinomial_Naive_Bayesian_0_1'
pickle.dump(model, open(filename,'wb'))